# All Monthly Expences
v.0.1 by Evgeny Polkovnikov and CoPilot

Answers to the question: Where my money go?

Operates on csv statement files from Bank of America Credit Card and First Horizon Checking. Save statements to Statements, and adjust your categories in expenses_cat.json

## Includes/Dependences

In [ ]:
# Extras (you may have to install these modules yourself)
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from itables import init_notebook_mode, show

In [ ]:
# System (already included with Python/Jupyter distro)
import json
import os
import re
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

## System Init

In [ ]:
init_notebook_mode(all_interactive=True) # Interactive tables
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.expand_frame_repr", False)

InteractiveShell.ast_node_interactivity = "none" # Don't print implicit last result

## User Data Setup
Below, adjust params to match your env.
Add you transaction files in csv format. Name them as YYYYMMDD_<Initials.Type.Bank>_<whatever>.csv
Currently supported - Bank of America (BOA), First Horizon (FHB)
Put your expense categories to the expenses_cats.json

In [ ]:
# Work dirs
transactions_dir='Statements/'
categories_file_path='expenses_cats.json'

## Pre-Calcs

In [ ]:
# Expense Categories
# Open the file and read the JSON data
imported_struct = {}
with open(categories_file_path, 'r') as cats_file:
    imported_struct = json.load(cats_file)

expense_categories = imported_struct['expense_categories']
monthly_expenses = imported_struct['monthly_expenses']
exlude_categories = imported_struct['exlude_categories']

In [ ]:
# Retrieve available transaction files
transactions_files_dict = {}
transactions_files_raw_list = [f for f in os.listdir(transactions_dir) if f.endswith('.csv')]

for file_name in transactions_files_raw_list:
    m = re.match(r'^(\d{8})_([a-zA-Z\.]+)_.*\.csv$', file_name)
    if m:
        date = m.group(1)
        bank = m.group(2)
        if date not in transactions_files_dict:
            transactions_files_dict[date] = {}
        if bank not in transactions_files_dict[date]:
            transactions_files_dict[date][bank] = os.path.join(transactions_dir, file_name)

transactions_dates_list = sorted(transactions_files_dict.keys(), reverse=True)

## Monthly Expenses Report

In [ ]:
# Prep the interactive report
# Consume and display data
@widgets.interact(trans_dd=widgets.Dropdown(
    options=transactions_dates_list,
    description='Transactions:',
    disabled=False,))
def report_monthly_expenses(trans_dd):
    month_transactions_files = transactions_files_dict[trans_dd]
    
    # Create empty pandas data_frame with following columns: Date, BankAcc, Payee, Amount, Category, isMonthly
    cdf_initial_columns = ['Date', 'BankAcc', 'TransName', 'Amount']
    cdf = pd.DataFrame(columns=cdf_initial_columns)
    
    # Read the Banks' CSV files into a Common DataFrame
    for bank in month_transactions_files:
        tdf = pd.DataFrame(columns=cdf_initial_columns)
        if 'BOA' in bank:
            bdf = pd.read_csv(month_transactions_files[bank])
            # Copy date from df to cdf mapping like this: Posted Date -> Date, Payee -> TransName, Amount -> Amount
            tdf['Date'] = bdf['Posted Date']
            tdf['BankAcc'] = bank
            tdf['TransName'] = bdf['Payee']
            tdf['Amount'] = bdf['Amount']
        elif 'FHB' in bank:
            bdf = pd.read_csv(month_transactions_files[bank])
            tdf['Date'] = bdf['Date']
            tdf['BankAcc'] = bank
            tdf['TransName'] = bdf['Description']
            # Add code to set tdf['Amount'] column cell value to either bdf['Debit'] or bdf['Credit'] depending on which one is not NaN.
            # If both are NaN, set tdf['Amount'] to 0.
            tdf['Amount'] = bdf['Debit'].fillna(0) + bdf['Credit'].fillna(0)
        
        if cdf.empty and not tdf.empty:
            cdf = tdf.copy()
        elif tdf.empty:
            pass
        else:
            cdf = pd.concat([cdf, tdf])

    # Add new column Category to the CommonDataFrame
    cdf['Category'] = "Uncat " + cdf['BankAcc'] + " " + cdf['TransName']
    for key in expense_categories:
        cdf.loc[cdf['TransName'].str.contains('|'.join(expense_categories[key]), case=False), 'Category'] = key
    
    # Exclde rows with Category in exlude_categories to avoid double counting
    cdf = cdf[~cdf['Category'].str.contains('|'.join(exlude_categories), case=False)]
    
    cdf['isMonthly'] = 'No'
    cdf.loc[cdf['Category'].str.contains('|'.join(monthly_expenses), case=False), 'isMonthly'] = 'Yes'
    
    # Create new dataframe with sum of Amount by Category, include rows with NaN Category
    cdf_sum = cdf.groupby('Category')['Amount'].sum().reset_index()
    # Add isMonthly column to cdf_sum
    cdf_sum['isMonthly'] = 'No'
    cdf_sum.loc[cdf_sum['Category'].str.contains('|'.join(monthly_expenses), case=False), 'isMonthly'] = 'Yes'
    # Sort by Amount in descending order
    cdf_sum = cdf_sum.sort_values(by='Category', ascending=True)
    
    # Sum all negative numbers from cdf_sum
    total_this_month = cdf_sum[cdf_sum['Amount'] < 0]['Amount'].sum()
    total_recurring_month = cdf_sum[(cdf_sum['Amount'] < 0) & (cdf_sum['isMonthly'] == 'Yes')]['Amount'].sum()
    total_subs = cdf[cdf['Category'].str.contains('Subscription')]['Amount'].sum()
    total_food = cdf[cdf['Category'].str.contains('Food')]['Amount'].sum()
    total_util = cdf[cdf['Category'].str.contains('Util')]['Amount'].sum()
    total_food_util = total_food + total_util
    car_gas = cdf[cdf['Category'].str.contains('Car Gas')]['Amount'].sum()
    total_car = (  car_gas
                 + cdf[cdf['Category'].str.contains('Car Maint')]['Amount'].sum()
                 + cdf[cdf['Category'].str.contains('Car Insurance')]['Amount'].sum())
    total_mandatory = (  total_food_util
                       + cdf[cdf['Category'].str.contains('Insurance')]['Amount'].sum()
                       + cdf[cdf['Category'].str.contains('HOA')]['Amount'].sum()
                       + cdf[cdf['Category'].str.contains('Car Gas')]['Amount'].sum()
                       + cdf[cdf['Category'].str.contains('Car Maint')]['Amount'].sum()
                       + cdf[cdf['Category'].str.contains('Doctors')]['Amount'].sum())
    total_non_mandatory = total_this_month - total_mandatory
    # Calculate total_wages as sum of cdf['Category'] containing word 'Wage'
    total_wages = cdf[cdf['Category'].str.contains('Wage')]['Amount'].sum()
    
    # Print total_this_month formatted as currency
    print(f"All Spending for {trans_dd}")

    month_summary_df = pd.DataFrame(columns=['Food', 'Util', 'Subs', 'Car Gas', 'Total Car', 'Total Mandatory',
                                             'Total Expenses', 'Total Wages', 'Balance'])
    month_summary_df.loc[trans_dd] = pd.Series({'Food': total_food,
                                                'Util': total_util,
                                                'Subs': total_subs,
                                                'Car Gas': car_gas,
                                                'Total Car': total_car,
                                                'Total Mandatory': total_mandatory,
                                                'Total Expenses': total_this_month,
                                                'Total Wages': total_wages,
                                                'Balance': total_wages + total_this_month})
    

    show(month_summary_df, "Month Summary", paging=False)
    print("* Total Mandatory = Food + Utilities + Insurance + Mortgage + HOA + Car + Doctors")

    
    # Display the Summary chart table
    cdf_sum_expenses_only = cdf_sum[cdf_sum['Amount'] < 0]
    cdf_sum_expenses_only.loc[:, 'Amount'] = cdf_sum_expenses_only['Amount'].abs()
     
    cdf_sum_expenses_only = cdf_sum_expenses_only.sort_values(by='Category', ascending=True)

    cdf_sum_expenses_only['GenCat'] = cdf_sum_expenses_only['Category']
    for gen_cat in ['Food', 'Util', 'Car', 'Insurance', 'Doctors', 'Subscription', 'House']:
        cdf_sum_expenses_only.loc[cdf_sum_expenses_only['Category'].str.contains(gen_cat), 'GenCat'] = gen_cat

    # Get a list of categories in the order they appear in cdf_sum_expenses_only
    #fig_cat_order = cdf_sum_expenses_only['Category'].tolist()
    # Get a list of values in the order they appear in cdf_sum_expenses_only
    #fig_val_order = cdf_sum_expenses_only['Amount'].tolist()
    gen_expenses_diagram = px.pie(data_frame=cdf_sum_expenses_only,
                 names='GenCat',
                 values='Amount',
                 # Add an order parameter to the px.pie call to sort the slices in the pie chart
                 #category_orders={'names': fig_cat_order}, # TODO: Does not work Python 3.10, plotly 6.0.0.
                 # Fix error: ColumnNotFoundError: The following columns were not found: ['b'] Hint: Did you mean one of these columns: ['Category', 'Amount']?
                 title='General Expense Distribution')
    gen_expenses_diagram.update_traces(textposition='inside', textinfo='percent+label', sort=False)
    gen_expenses_diagram.show()

    # Display the Summary table
    show(cdf_sum[['Category', 'isMonthly', 'Amount']], "Expenses Summary", paging=False)
    # Display the details table
    show(cdf[['Date', 'BankAcc', 'TransName', 'Amount', 'Category', 'isMonthly']], "All Details", paging=False)

print("End of the Monthly Expenses Report")